In [1]:
# import libraries
import numpy as np
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from config_db import password_db
from flask import Flask, jsonify
import pandas as pd
import json

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [2]:
connection_string = 'postgres:'+password_db+'@localhost:5432/travel_data'

engine=create_engine(f"postgresql://{connection_string}")

In [3]:
Base = automap_base()

In [4]:
Base.prepare(engine, reflect = True)

In [5]:
travel_details = Base.classes.travel_data_details()
# Station = Base.classes.station

In [6]:
travel_details

In [7]:
Travel_Details = Base.classes.travel_data_details
Travel_Details.__dict__

mappingproxy({'__table__': Table('travel_data_details', MetaData(bind=None), Column('index', INTEGER(), table=<travel_data_details>, primary_key=True, nullable=False), Column('date', VARCHAR(length=12), table=<travel_data_details>), Column('state_fips', INTEGER(), table=<travel_data_details>), Column('state_code', VARCHAR(length=2), table=<travel_data_details>), Column('pop_stay_at_home', INTEGER(), table=<travel_data_details>), Column('pop_not_stay_at_home', INTEGER(), table=<travel_data_details>), Column('trips', INTEGER(), table=<travel_data_details>), Column('trips_1', INTEGER(), table=<travel_data_details>), Column('trips_1_3', INTEGER(), table=<travel_data_details>), Column('trips_3_5', INTEGER(), table=<travel_data_details>), Column('trips_5_10', INTEGER(), table=<travel_data_details>), Column('trips_10_25', INTEGER(), table=<travel_data_details>), Column('trips_25_50', INTEGER(), table=<travel_data_details>), Column('trips_50_100', INTEGER(), table=<travel_data_details>), Colum

In [8]:
session = Session(engine)

In [9]:
# test sql
connection = engine.connect()

In [10]:
test_travel_df = pd.read_sql("""select year, month, month_abbr, month_year, state, 
round(avg(pop_stay_at_home),0) as no_travel_pop, round(avg(pop_not_stay_at_home),0) as travel_pop, 
round(avg(trips),0) as total_trips, round(avg(trips_1),0) as trips_1, round(avg(trips_1_3),0) as trips_1_3,
round(avg(trips_3_5),0) as trips_3_5, round(avg(trips_5_10),0) as trips_5_10, round(avg(trips_10_25),0) as trips_10_25,
round(avg(trips_25_50),0) as trips_25_50, round(avg(trips_50_100),0) as trips_50_100,
round(avg(trips_100_250),0) as trips_100_250, round(avg(trips_250_500),0) as trips_250_500,
round(avg(trips_500),0) as trips_500 from travel_data_details 
GROUP BY state, month_year, year, month, month_abbr
order by state, year, month, month_year, month_abbr """, connection)
test_travel_df
    # convert pandas dataframe to json
# test_travel_json = test_travel_df.to_json()
# test_travel_json

,year,month,month_abbr,month_year,state,no_travel_pop,travel_pop,total_trips,trips_1,trips_1_3,trips_3_5,trips_5_10,trips_10_25,trips_25_50,trips_50_100,trips_100_250,trips_250_500,trips_500
0,2019,1,Jan,Jan-2019,Alabama,817303.0,4055631.0,15694066.0,3457789.0,3928199.0,2102792.0,2550304.0,2381688.0,803007.0,297972.0,133914.0,29997.0,8404.0
1,2019,10,Oct,Oct-2019,Alabama,873602.0,3999332.0,17122849.0,3509941.0,4069888.0,2273704.0,2922332.0,3032815.0,969267.0,256227.0,67312.0,15779.0,5584.0
2,2019,11,Nov,Nov-2019,Alabama,863024.0,4009910.0,16051347.0,3222625.0,3741107.0,2160788.0,2737225.0,2880013.0,957496.0,266789.0,62945.0,16703.0,5656.0
3,2019,12,Dec,Dec-2019,Alabama,885949.0,3986985.0,16224584.0,3112266.0,3782532.0,2224508.0,2823810.0,2944228.0,972062.0,271608.0,68939.0,18240.0,6391.0
4,2019,2,Feb,Feb-2019,Alabama,786528.0,4086406.0,15782998.0,3406372.0,3917605.0,2100610.0,2574130.0,2433995.0,838147.0,321631.0,147557.0,31687.0,11265.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,2020,5,May,May-2020,Wyoming,138418.0,439319.0,2161415.0,629094.0,590282.0,274332.0,214916.0,185114.0,145797.0,79799.0,36454.0,4334.0,1294.0
1046,2020,6,Jun,Jun-2020,Wyoming,130489.0,447248.0,2111551.0,600523.0,552089.0,259166.0,208627.0,190782.0,155855.0,90332.0,46482.0,5833.0,1863.0
1047,2020,7,Jul,Jul-2020,Wyoming,141108.0,436629.0,1887541.0,509499.0,491375.0,228916.0,190580.0,182780.0,146981.0,84735.0,44631.0,5770.0,2275.0
1048,2020,8,Aug,Aug-2020,Wyoming,139191.0,438546.0,1892930.0,490849.0,500760.0,232761.0,192435.0,182371.0,141622.0,90769.0,50868.0,6887.0,3608.0


In [11]:
#     convert pandas dataframe to json -- discard
# test_travel_json = test_travel_df.to_json()
# test_travel_json

In [12]:
test_travel_json1 = json.dumps(test_travel_df.to_dict('records'))
test_travel_json1

'[{"year": "2019", "month": "1", "month_abbr": "Jan", "month_year": "Jan-2019", "state": "Alabama", "no_travel_pop": 817303.0, "travel_pop": 4055631.0, "total_trips": 15694066.0, "trips_1": 3457789.0, "trips_1_3": 3928199.0, "trips_3_5": 2102792.0, "trips_5_10": 2550304.0, "trips_10_25": 2381688.0, "trips_25_50": 803007.0, "trips_50_100": 297972.0, "trips_100_250": 133914.0, "trips_250_500": 29997.0, "trips_500": 8404.0}, {"year": "2019", "month": "10", "month_abbr": "Oct", "month_year": "Oct-2019", "state": "Alabama", "no_travel_pop": 873602.0, "travel_pop": 3999332.0, "total_trips": 17122849.0, "trips_1": 3509941.0, "trips_1_3": 4069888.0, "trips_3_5": 2273704.0, "trips_5_10": 2922332.0, "trips_10_25": 3032815.0, "trips_25_50": 969267.0, "trips_50_100": 256227.0, "trips_100_250": 67312.0, "trips_250_500": 15779.0, "trips_500": 5584.0}, {"year": "2019", "month": "11", "month_abbr": "Nov", "month_year": "Nov-2019", "state": "Alabama", "no_travel_pop": 863024.0, "travel_pop": 4009910.0,

In [13]:
# # for row in session.query(Travel_Details.month, Travel_Details.state,Travel_Details.pop_stay_at_home):
# #     print(row)

# no_travel = session.query(Travel_Details.year, Travel_Details.month, Travel_Details.state,func.avg(Travel_Details.pop_stay_at_home)).\
# group_by(Travel_Details.year, Travel_Details.month,Travel_Details.state).\
# order_by(Travel_Details.year, Travel_Details.state,Travel_Details.month).all()

# #     no_travel = session.query(Travel_Details.month, Travel_Details.state,func.avg(Travel_Details.pop_stay_at_home)).\
# #     group_by(Travel_Details.month,Travel_Details.state).\
# #     order_by(Travel_Details.state,Travel_Details.month).all()

# no_travel
    


In [14]:
# no_travel = [(int(no_travel_tuple[0]), int(no_travel_tuple[1]), no_travel_tuple[2], round(float(no_travel_tuple[3])))\
#              for no_travel_tuple in no_travel]
# no_travel

In [15]:
# def row2dict(row):
#     d = {}
#     for column in row.__table__.columns:
#         d[column.name] = str(getattr(row, column.name))

#     return d

In [16]:
# session=Session(engine)

In [17]:
# geomap_travel = session.query(Travel_Details.year, Travel_Details.month, Travel_Details.state, Travel_Details.latitude,\
#                               Travel_Details.longitude, func.avg(Travel_Details.pop_not_stay_at_home)).\
#                               group_by(Travel_Details.year, Travel_Details.month,Travel_Details.state, \
#                                        Travel_Details.latitude, Travel_Details.longitude).\
#                               order_by(Travel_Details.year, Travel_Details.state,Travel_Details.month, \
#                                        Travel_Details.latitude, Travel_Details.longitude).all()
# geomap_travel

In [18]:
# geomap_travel = [(int(gp_travel_tuple[0]), int(gp_travel_tuple[1]), gp_travel_tuple[2], gp_travel_tuple[3], \
#                   gp_travel_tuple[4], round(float(gp_travel_tuple[5]))) for gp_travel_tuple in geomap_travel]
# geomap_travel